## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Busselton,AU,-33.6500,115.3333,53.58,83,95,4.29,overcast clouds
1,1,Severo-Kurilsk,RU,50.6789,156.1250,41.56,79,86,13.62,overcast clouds
2,2,Rikitea,PF,-23.1203,-134.9692,75.13,85,84,21.00,light rain
3,3,Mataura,NZ,-46.1927,168.8643,48.25,97,100,14.07,light rain
4,4,Seymchan,RU,62.8833,152.4333,5.76,83,100,7.16,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? (F) "))
max_temp = float(input("What is the maximum temperature you would like for your trip? (F) "))

What is the minimum temperature you would like for your trip? (F) 75
What is the maximum temperature you would like for your trip? (F) 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,75.13,85,84,21.00,light rain
7,7,Cabo San Lucas,MX,22.8909,-109.9124,88.52,71,20,11.99,few clouds
10,10,Kapaa,US,22.0752,-159.3190,81.48,85,40,1.99,scattered clouds
17,17,Pontianak,ID,-0.0333,109.3333,76.98,94,75,2.30,broken clouds
22,22,Dingle,PH,10.9995,122.6711,77.36,88,95,7.96,overcast clouds
23,23,Lazaro Cardenas,MX,17.9583,-102.2000,82.18,76,1,4.12,clear sky
25,25,Mabaruma,GY,8.2000,-59.7833,75.16,92,94,3.06,overcast clouds
26,26,Hilo,US,19.7297,-155.0900,82.26,70,40,8.05,scattered clouds
28,28,Oriximina,BR,-1.7656,-55.8661,81.45,69,39,3.58,heavy intensity rain
30,30,Atuona,PF,-9.8000,-139.0333,77.92,67,5,11.95,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Rikitea,PF,-23.1203,-134.9692,75.13,85,84,21.00,light rain
7,7,Cabo San Lucas,MX,22.8909,-109.9124,88.52,71,20,11.99,few clouds
10,10,Kapaa,US,22.0752,-159.3190,81.48,85,40,1.99,scattered clouds
17,17,Pontianak,ID,-0.0333,109.3333,76.98,94,75,2.30,broken clouds
22,22,Dingle,PH,10.9995,122.6711,77.36,88,95,7.96,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Rikitea,PF,75.13,light rain,-23.1203,-134.9692,
7,Cabo San Lucas,MX,88.52,few clouds,22.8909,-109.9124,
10,Kapaa,US,81.48,scattered clouds,22.0752,-159.3190,
17,Pontianak,ID,76.98,broken clouds,-0.0333,109.3333,
22,Dingle,PH,77.36,overcast clouds,10.9995,122.6711,
23,Lazaro Cardenas,MX,82.18,clear sky,17.9583,-102.2000,
25,Mabaruma,GY,75.16,overcast clouds,8.2000,-59.7833,
26,Hilo,US,82.26,scattered clouds,19.7297,-155.0900,
28,Oriximina,BR,81.45,heavy intensity rain,-1.7656,-55.8661,
30,Atuona,PF,77.92,clear sky,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat}, {lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
for index, row in hotel_df.iterrows():
    hotel_df.replace("", nan_value, inplace=True)

clean_hotel_df = hotel_df.dropna()


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))